# closed-world

In [ ]:
import os  #closed-world
import csv
import itertools
import numpy as np
# Define the folder path
folder_path = r'D:\packetsizecw'

def load_data(directory):
    """
    Load data from ascii files
    """
    X, y = [], []
    class_counter = 0
    for root, dirs, files in os.walk(directory):
        for fname in files:
            cls, inst  = fname.split('-')
            trace_class = int(cls)
            file = open(os.path.join(root, fname), 'r')
            print(file)
            packetsize = []
            for line in file:
                pieces = line.strip("\n").split(':')
                if int(pieces[1]) == 0:
                    break
                length = int(pieces[1])
                packetsize.append(length)
        
            feature=extract(packetsize)
            X.append(feature)
            y.append(trace_class)

    # wrap as numpy array
    X, Y = np.array(X), np.array(y)
    return X, Y

# Define the feature extraction function
def extract(sinste):  #sinste: list of packet sizes
    
    #first 4 features
    features = []
            
    total = []
    cum = []
    pos = []
    neg = []
    inSize = 0
    outSize = 0
    inCount = 0
    outCount = 0

    for i in range(0, len(sinste)):
        # incoming packets
        if sinste[i]  > 0:
            inSize += sinste[i] 
            inCount += 1
            # cumulated packetsizes
            if len(cum) == 0:
                cum.append(sinste[i] )
                total.append(sinste[i] )
                pos.append(sinste[i] )
                neg.append(0)
            else:
                cum.append(cum[-1] + sinste[i] )
                total.append(total[-1] + abs(sinste[i] ))
                pos.append(pos[-1] + sinste[i] )
                neg.append(neg[-1] + 0)

        # outgoing packets
        if sinste[i]  < 0:
            outSize += abs(sinste[i] )
            outCount += 1
            if len(cum) == 0:
                cum.append(sinste[i] )
                total.append(abs(sinste[i] ))
                pos.append(0)
                neg.append(abs(sinste[i] ))
            else:
                cum.append(cum[-1] + sinste[i] )
                total.append(total[-1] + abs(sinste[i] ))
                pos.append(pos[-1] + 0)
                neg.append(neg[-1] + abs(sinste[i] ))

    # add feature
    features.append(inCount)
    features.append(outCount)
    features.append(outSize)
    features.append(inSize)
    cumFeatures = np.interp(np.linspace(total[0], total[-1], 101), total, cum)
    for el in itertools.islice(cumFeatures, 1, None):
        features.append(el)
    return features
X,y=load_data(folder_path)

In [ ]:
from sklearn import svm  #cw
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import confusion_matrix

# Assume X and y are loaded and preprocessed data

# Scale the features to the range [-1, 1]
scaler = MinMaxScaler(feature_range=(-1, 1))
X_scaled = scaler.fit_transform(X)

# Split the scaled data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2)

# label_encoder = LabelEncoder()
# y_train = label_encoder.fit_transform(y_train)
# y_test = label_encoder.transform(y_test)

# Create an SVM classifier object
clf = svm.SVC(kernel='rbf')

# Define the parameter grid for grid search
param_grid = {'C': [2048, 4096, 8192, 16384, 32768, 65536, 131072], 'gamma': [0.125, 0.25, 0.5, 1, 2, 4, 8]}

# Create a GridSearchCV object
grid_search = GridSearchCV(clf, param_grid, n_jobs=8, verbose=1)

# Fit the grid search to the training data
grid_search.fit(X_train, y_train)

# Get the best parameters from the grid search
best_parameters = grid_search.best_estimator_.get_params()

print("Best parameters: ", best_parameters)

# Create a new SVM classifier with the best parameters
svm_model = svm.SVC(kernel='rbf', C=best_parameters['C'], gamma=best_parameters['gamma'])

# Train the SVM model on the training data
svm_model.fit(X_train, y_train)

# Use the trained classifier to predict labels for the test data
y_pred = svm_model.predict(X_test)

# Evaluate the accuracy of the classifier
accuracy = accuracy_score(y_test, y_pred)
print('Accuracy:', accuracy)

In [ ]:
from sklearn import svm  #ten-fold cross-validation
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import GridSearchCV

# Assume X and y are loaded and preprocessed data

# Scale the features to the range [-1, 1]
scaler = MinMaxScaler(feature_range=(-1, 1))
X_scaled = scaler.fit_transform(X)

# Split the scaled data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2)

# Create an SVM classifier object
clf = svm.SVC(kernel='rbf')

# Define the parameter grid for grid search
param_grid = {'C': [2048, 4096, 8192, 16384, 32768, 65536, 131072], 'gamma': [0.125, 0.25, 0.5, 1, 2, 4, 8]}

# Create a GridSearchCV object
grid_search = GridSearchCV(clf, param_grid, n_jobs=8, verbose=1)

# Fit the grid search to the training data
grid_search.fit(X_train, y_train)

# Get the best parameters from the grid search
best_parameters = grid_search.best_estimator_.get_params()

print("Best parameters: ", best_parameters)

# Create a new SVM classifier with the best parameters
svm_model = svm.SVC(kernel='rbf', C=best_parameters['C'], gamma=best_parameters['gamma'])

# Evaluate the SVM classifier using 10-fold cross-validation
scores = cross_val_score(svm_model, X_train, y_train, cv=10, n_jobs=8)

print("Cross-validation scores:", scores)
print("Mean score:", scores.mean())

# open-world

In [ ]:
import os  #open-world
import csv
import itertools
import numpy as np
# Define the folder path
mon_path=r'D:\packetsizecw'
unmon_path=r'D:\packetsizeow'

def load_mondata(directory):
    """
    Load data from monitored websites
    """
    X, y = [], []
    class_counter = 0
    for root, dirs, files in os.walk(directory):
#         if not unmon:
#         files = [fname for fname in files if len(fname.split('-')) == 2]
        for fname in files:
            cls, inst  = fname.split('-')
            trace_class = int(cls)
            file = open(os.path.join(root, fname), 'r')
            print(file)
            packetsize = []
            for line in file:
                pieces = line.strip("\n").split(':')
                if int(pieces[1]) == 0:
                    break
                length = int(pieces[1])
                packetsize.append(length)
                
            feature=extract(packetsize)
            X.append(feature)
            y.append(trace_class)

    # wrap as numpy array
    X, Y = np.array(X), np.array(y)
    return X, Y

def load_unmondata(directory):
    """
    Load data from unmonitored websites
    """
    X, y = [], []
    class_counter = 0
    for root, dirs, files in os.walk(directory):
#         if not unmon:
#         files = [fname for fname in files if len(fname.split('-')) == 2]
        for fname in files:
#             cls, inst  = fname.split('-')
#             trace_class = int(cls)
            file = open(os.path.join(root, fname), 'r')
            print(file)
            packetsize = []
            for line in file:
                pieces = line.strip("\n").split(':')
                if int(pieces[1]) == 0:
                    break
                length = int(pieces[1])
                packetsize.append(length)
        
            feature=extract(packetsize)
            X.append(feature)
            y=[100]*len(files)

    # wrap as numpy array
    X, Y = np.array(X), np.array(y)
    return X, Y

# Define the feature extraction function
def extract(sinste): #sinste: list of packet sizes
    features = []
    #first 4 features    
    total = []
    cum = []
    pos = []
    neg = []
    inSize = 0
    outSize = 0
    inCount = 0
    outCount = 0

    for i in range(0, len(sinste)):
        # incoming packets
        if sinste[i]  > 0:
            inSize += sinste[i] 
            inCount += 1
            # cumulated packetsizes
            if len(cum) == 0:
                cum.append(sinste[i] )
                total.append(sinste[i] )
                pos.append(sinste[i] )
                neg.append(0)
            else:
                cum.append(cum[-1] + sinste[i] )
                total.append(total[-1] + abs(sinste[i] ))
                pos.append(pos[-1] + sinste[i] )
                neg.append(neg[-1] + 0)

        # outgoing packets
        if sinste[i]  < 0:
            outSize += abs(sinste[i] )
            outCount += 1
            if len(cum) == 0:
                cum.append(sinste[i] )
                total.append(abs(sinste[i] ))
                pos.append(0)
                neg.append(abs(sinste[i] ))
            else:
                cum.append(cum[-1] + sinste[i] )
                total.append(total[-1] + abs(sinste[i] ))
                pos.append(pos[-1] + 0)
                neg.append(neg[-1] + abs(sinste[i] ))

    # add feature
    features.append(inCount)
    features.append(outCount)
    features.append(outSize)
    features.append(inSize)
    cumFeatures = np.interp(np.linspace(total[0], total[-1], 101), total, cum)
    for el in itertools.islice(cumFeatures, 1, None):
        features.append(el)
    return features

X_mon,y_mon=load_mondata(mon_path)
X_unmon,y_unmon=load_unmondata(unmon_path)

In [ ]:
sr=0.8
X_mon_tr = X_mon[:int(X_mon.shape[0]*sr)]
y_mon_tr = y_mon[:int(X_mon.shape[0]*sr)]
X_unmon_tr = X_unmon[:1000]
y_unmon_tr = y_unmon[:1000]

X_mon_te = X_mon[int(X_mon.shape[0]*sr):]
y_mon_te = y_mon[int(X_mon.shape[0]*sr):]
X_unmon_te = X_unmon[1000:]
y_unmon_te = y_unmon[1000:]

X_train = np.concatenate([X_mon_tr, X_unmon_tr])
y_train = np.concatenate([y_mon_tr, y_unmon_tr])
X_test=np.concatenate([X_mon_te, X_unmon_te])
y_test=np.concatenate([y_mon_te, y_unmon_te])

In [ ]:
from sklearn import svm
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.model_selection import GridSearchCV

# Assume X and y are loaded and preprocessed data

# Scale the features to the range [-1, 1]
scaler = MinMaxScaler(feature_range=(-1, 1))
# X_scaled = scaler.fit_transform(X)

# Split the scaled data into training and testing sets
# X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2)
X_train=scaler.fit_transform(X_train)
X_test=scaler.fit_transform(X_test)
# label_encoder = LabelEncoder()
# y_train = label_encoder.fit_transform(y_train)
# y_test = label_encoder.transform(y_test)

# Create an SVM classifier object
clf = svm.SVC(kernel='rbf')

# Define the parameter grid for grid search
param_grid = {'C': [2048, 4096, 8192, 16384, 32768, 65536, 131072], 'gamma': [0.125, 0.25, 0.5, 1, 2, 4, 8]}

# Create a GridSearchCV object
grid_search = GridSearchCV(clf, param_grid, n_jobs=8, verbose=1)

# Fit the grid search to the training data
grid_search.fit(X_train, y_train)

# Get the best parameters from the grid search
best_parameters = grid_search.best_estimator_.get_params()

print("Best parameters: ", best_parameters)

# Create a new SVM classifier with the best parameters
svm_model = svm.SVC(kernel='rbf', C=best_parameters['C'], gamma=best_parameters['gamma'])

# Train the SVM model on the training data
svm_model.fit(X_train, y_train)

# Use the trained classifier to predict labels for the test data
y_pred = svm_model.predict(X_test)

# Evaluate the accuracy, precision, and recall of the classifier
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='weighted')
recall = recall_score(y_test, y_pred, average='weighted')
print('Accuracy:', accuracy)
print('Precision:', precision)
print('Recall:', recall)

# Calculate confusion matrix
cm = confusion_matrix(y_test, y_pred)

# Calculate TPR and FPR for each class
num_classes = len(label_encoder.classes_)
tpr_list = []
fpr_list = []
for i in range(num_classes):
    tp = cm[i,i]
    fp = np.sum(cm[:,i]) - tp
    fn = np.sum(cm[i,:]) - tp
    tn = np.sum(cm) - tp - fp - fn
    
    tpr = tp / (tp + fn)
    fpr = fp / (fp + tn)
    
    tpr_list.append(tpr)
    fpr_list.append(fpr)
    
    print(f"Class {i} - TPR: {tpr:.4f}, FPR: {fpr:.4f}")

# Calculate macro-average TPR and FPR
macro_tpr = np.mean(tpr_list)
macro_fpr = np.mean(fpr_list)
print(f"Macro-average TPR: {macro_tpr:.4f}, FPR: {macro_fpr:.4f}")